# Export image function notebook within ArcGIS Pro


## Notebook for Exporting Jpeg Images from NICFI, Sentinel Datasource, Esri Source.

### V2:
- Export the Esri source `World Imagery` 
- Based on the certain shapefile
- Scall setting: 
  - `< 4ha` fixed setting 5ha
  - `> 4ha` dynamic setting 2times of the shape
- Label infor in the jpeg: index, image date( from the basedmap `World Imagery` ), 
- Boundary line of the shape on the jpeg
- Only export the last imagery of the Esri world imagery


### V1:
- Target shapefile: 1823_ADRSM
- Important fields: `DEN_BOT`, `AREA_HA`, `Index`, `SHAPE@`
- Export only if `AREA_HA` > 10
- NICFI datasource imported via GEE in Python notebook
- Sentinel-2 data could be imported through several channels; this solution utilizes ArcGIS Pro Living Atlas (AWS host)
- Dynamic scale properties:
  - If size < 10ha, image will scale at 8 times 
  - If 10ha < size < 50ha, image will scale at 5 times
  - If 50ha < shape size < 100ha, image will scale at 2 times
  - If shape size > 100ha, image will scale at 1.3 times

- Five Years and 60 images for each dumpsite using NICFI  
1. Each block contains the function description
2. NICFI:
   - Get NICFI
   - Add NICFI
   - Determine the date range (check the new update)
   - Get Project, layout, map, mapview, mapframe
   - Processing other Exporting 
3. Sentinel: 
   - Add sentinel data from Living Atlas (or other resource option)
4. Export multiple date's images (5 years x 12 months) of each shape

### Environment:
- ArcGIS Pro 3.2.2
- Python 3.8.19


## Basic Function
Run basic before export.

### Parameter setting

In [51]:
# Shapefile layer name in arcgis
shapefile_layer_name = "c1823_ADRSM_01ha"

# export jpeg path and folder
export_path = r"C:\Users\lycaz\Desktop\world_imagery"

# export jpeg resolution
export_resolution = 300

# minimum  area size of the shapefile 
min_area = 0.1

# lib import 
import arcpy
import datetime
# check the shapefile return selected count of feature class
print(f"Count of shape of {shapefile_layer_name} : {arcpy.GetCount_management(shapefile_layer_name)}") # if selected 1 it will return 1

current_time = datetime.datetime.now()
print("Current Time:", current_time)

Count of shape of c1823_ADRSM_01ha : 1206
Current Time: 2024-07-25 18:32:48.610189


#### Basic object fetched

In [52]:
# Get Project, layout, map, mapview, mapframe
# Before using this code , you should open and load both layout window and Map window, or it will erro: mapview is None

current_project=arcpy.mp.ArcGISProject("CURRENT")
layout=current_project.listLayouts()[1]  # 0 is the dataframe drive layout(text,area) # 1 is the 
amap=current_project.listMaps()[0]  
mv=current_project.activeView    #mv=amap.defaultView
mf= layout.listElements("MAPFRAME_ELEMENT")[0]  #get the mapframe in the layout

print("Current layout name:",layout.name)
print("Mapview.extent: ", mv.camera.getExtent())
print("Mapname: ",amap.name)
print("Mfname: ",mf.name)
current_time = datetime.datetime.now()
print("Current Time:", current_time)

Current layout name: JpegExport
Mapview.extent:  -8154364.42271678 -414167.970613891 -8154145.25723675 -413872.965859518 NaN NaN NaN NaN
Mapname:  Map
Mfname:  Map Frame
Current Time: 2024-07-25 18:32:50.289188


## Exporting func

#### Get Imagery date function

In [53]:
import requests

def get_imagery_date(lat, lon):
    # URL for the ESRI World Imagery REST endpoint
    url = "https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/identify"
    
    # Parameters for the identify operation
    params = {
        'geometry': f'{lon},{lat}',
        'geometryType': 'esriGeometryPoint',
        'sr': 4326,
        'layers': 'all',
        'tolerance': 2,
        'mapExtent': f'{lon-0.01},{lat-0.01},{lon+0.01},{lat+0.01}',
        'imageDisplay': '800,600,96',
        'returnGeometry': False,
        'f': 'json'
    }
    
    # Make the request
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extract and print relevant metadata
    if 'results' in data and len(data['results']) > 0:
        for result in data['results']:
            if 'attributes' in result:
                attributes = result['attributes']
                # Extract specific metadata fields (e.g., acquisition date, source, etc.)
                if 'AcquisitionDate' in attributes:
                    print(f"Acquisition Date: {attributes['AcquisitionDate']}")
                if 'Source' in attributes:
                    print(f"Source: {attributes['Source']}")
                if 'DATE (YYYYMMDD)' in attributes:
                    if attributes['DATE (YYYYMMDD)'] is not None:
                        return attributes['DATE (YYYYMMDD)']
                    print(f"Date: {attributes['DATE (YYYYMMDD)']}")
                # Print all available attributes for inspection
                
    else:
        print("No imagery information found for the specified location.")


def get_lat_long(camera):
    # Get the center of the map view
    # Calculate the center point of the extent
    extent = camera.getExtent()
    center_x = (extent.XMin + extent.XMax) / 2
    center_y = (extent.YMin + extent.YMax) / 2
    
    # Create a point geometry for the center point
    center_point = arcpy.PointGeometry(arcpy.Point(center_x, center_y), extent.spatialReference)
    
    # Define the target spatial reference (WGS84)
    target_sr = arcpy.SpatialReference(4326)
    
    # Project the center point to WGS84
    center_point_wgs84 = center_point.projectAs(target_sr)
    
    # Extract the coordinates in decimal degrees
    latitude = center_point_wgs84.centroid.Y
    longitude = center_point_wgs84.centroid.X
    return latitude, longitude

#### Export function

In [56]:
# this function is for export the images 
# it can be call onetime or iterate all in different year and month datasource set
import os

#only based on the shape "1823_ADRSM" and the 
# default parameter
fc = shapefile_layer_name
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]
default_path=r"C:\Users\lycaz\Desktop\output"
file_type='jpg'
min_area=min_area


def export_image(path=default_path,src_label="EsriWorldImagery",dpi=150,f_type=file_type,istest=False):
    """
    Brief description of the function.

    Parameters:

    time (str): Description of param2.
    path(str): which folder you want to saved
    
    Returns:
    type: Description of what the function returns.
    """
    print(f"config:, path:{path}, label:{src_label}, dpi={dpi},f_type{f_type}")
    test_count=50 # only for test
    
    if os.path.exists(path)==False:
        print("Path does not exist.")
        return
    
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        count_im=0
        for row in cursor:
            #print(u'{0}, {1},{2}'.format(row[2],row[0], row[1]))
            
            print(f" Processing index {row[2]} {row[0]}", end="\r")
            if row[1] > min_area: 
                count_im+=1
                print(f"image coount {count_im}", end="\r")
                geometry=row[3]
                mv.camera.setExtent(geometry.extent)
                amap.defaultCamera=mv.camera
                amap.defaultCamera.scale*=2
                mf.camera.setExtent(amap.defaultCamera.getExtent())
                lati,longi=get_lat_long(mf.camera)
                date_stamp=get_imagery_date(lati,longi)
                print(f"Current Lat:{lati}, Long:{longi}, Date:{date_stamp}")
                if row[1]<4:
                    desired_scale=5/row[1]
                    mf.camera.scale*=desired_scale
                else:
                    mf.camera.scale*=2
                ele=layout.listElements(wildcard="Text")[0]
                ele.text=f"{row[2]}-{date_stamp}-{row[1]:.2f}ha"
                ele_source=layout.listElements(wildcard="Text 1")[0]
                ele_source.text=src_label
                if f_type=='jpg':
                    layout.exportToJPEG(f"{path}\\{str(row[2])}-{date_stamp}-{src_label}.jpg",resolution=dpi)
                elif f_type=="tif":
                    layout.exportToTIFF(f"{path}\\{str(row[2])}-{date_stamp}-{src_label}.tif") # without geotif

            test_count-=1
            if istest==True and test_count==0:
                break
                    
                
        print('\n')
        print(f"Complete current the image in: {time} ")
    
    
print(f"Exporting function ready path:{default_path}")   
    
current_time = datetime.datetime.now()
print("Current Time:", current_time)

Exporting function ready path:C:\Users\lycaz\Desktop\output
Current Time: 2024-07-25 18:34:48.399833


## Excute export all the shapes' images

In [57]:

# Before exporting, confirm the layout is setting well, the tile map is loaded and the shapefile is outlined.


# get the certain shape of shapefile  and iterate the shapefile
# handle the mv (mapview), do not effect the layout 
import time 
start_time = time.time()

fc = shapefile_layer_name
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]

path=r"C:\Users\lycaz\Desktop\output"    # schange it before exporting !!!!!!
datasource_str="worldimagery" 

# call the exporting function for all the shapes in the shapefile
export_image(path,datasource_str,150,"jpg",False)

stop_time = time.time()
elapsed_time = stop_time - start_time

print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")
current_time = datetime.datetime.now()
print("Current Time:", current_time)

config:, path:C:\Users\lycaz\Desktop\output, label:worldimagery, dpi=150,f_typejpg
Current Lat:-12.135973026688887, Long:-75.24903611262647, Date:20211119
Current Lat:-6.2485940006078735, Long:-79.66107437972877, Date:20230102
Current Lat:-7.807486331512859, Long:-77.64293549737525, Date:20220824
Current Lat:-12.8345802929423, Long:-76.02301579514277, Date:20221203
Current Lat:-6.725945615987959, Long:-76.25526990393287, Date:20180729
Current Lat:-10.00543356495644, Long:-77.13246042450893, Date:20210528
Current Lat:-13.58234304386884, Long:-71.41688854561986, Date:20170725
Current Lat:-17.07857656100099, Long:-70.86770450835633, Date:20221021
Current Lat:-12.642937348328381, Long:-73.07202255649574, Date:20210928
Current Lat:-15.092733476911372, Long:-69.8173815800958, Date:20210821
Current Lat:-12.751728369446727, Long:-74.12661839492296, Date:20190606
Current Lat:-1.7893522049580077, Long:-73.40783559133942, Date:20190510ro
Current Lat:-13.2021648904659, Long:-70.68698732825257, Dat

### Test the export function

In [36]:
import time
import arcpy

stat_time = time.time()

fc=shapefile_layer_name
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]
path=r"C:\Users\lycaz\Desktop\output"    
data_source="wordimagery" 
time_stamp="202403"
export_image(path,data_source,150,"jpg",True)


end_time = time.time()
elapsed_time = end_time - stat_time

print(f"Start time: {time.ctime(stat_time)}")
print(f"Stop time: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

config:, path:C:\Users\lycaz\Desktop\output, label:wordimagery, dpi=150,f_typejpg
image coount 1dex 50 Botadero Purupampa

Complete current the image in: 202403 
Start time: Thu Jul 25 16:48:40 2024
Stop time: Thu Jul 25 16:48:43 2024
Elapsed time: 2.21 seconds


#### Test the get_date function


In [44]:

# Example usage
latitude = -10.09337942  # Replace with your latitude
longitude = -76.6385212  # Replace with your longitude
print("date:",get_imagery_date(latitude, longitude))

import arcpy

# Get the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Get the active map view
map_view = aprx.activeView

# Check if there's an active view
if map_view:
    # Get the camera object
    camera = map_view.camera
    
    # Get the camera's extent
    extent = camera.getExtent()
    
    # Calculate the center point of the extent
    center_x = (extent.XMin + extent.XMax) / 2
    center_y = (extent.YMin + extent.YMax) / 2
    
    # Create a point geometry for the center point
    center_point = arcpy.PointGeometry(arcpy.Point(center_x, center_y), extent.spatialReference)
    
    # Define the target spatial reference (WGS84)
    target_sr = arcpy.SpatialReference(4326)
    
    # Project the center point to WGS84
    center_point_wgs84 = center_point.projectAs(target_sr)
    
    # Extract the coordinates in decimal degrees
    latitude = center_point_wgs84.centroid.Y
    longitude = center_point_wgs84.centroid.X
    print("date:",get_imagery_date(latitude, longitude))
    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print("No active map view found.")

date: 20210809
date: 20220816
Latitude: -11.366217971175205, Longitude: -76.60685810084154
